In [ ]:
import os
import shutil

def find_and_copy_fits_files(src_folder):
    # Create the target directory if it doesn't exist
    target_folder = os.path.join(src_folder, 'm101_only')
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Walk through all the folders within the source folder
    for root, dirs, files in os.walk(src_folder):
        for file in files:
            if file.lower().endswith('.fit') and ('m101' in file.lower()):
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(target_folder, file)
                
                # Copy the file to the target directory
                shutil.copy2(source_file_path, destination_file_path)
                print(f'Copied {file} to {target_folder}')

# Define the source folder
source_folder = 'D:\Astronomy-Image-Processing\m51\2024-07-26+27'

# Call the function
find_and_copy_fits_files(source_folder)


In [ ]:
import numpy as np
import os
from astropy.io import fits
from astropy.nddata import CCDData
import ccdproc
import astroalign as aa
from astroscrappy import detect_cosmics
import re

def load_all_fits_images(root_folder):
    """Recursively load all FITS files from a specified root folder and return both the HDUList and filenames."""
    files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for f in filenames:
            if f.endswith('.fit'):
                files.append(os.path.join(dirpath, f))
    files.sort(key=lambda x: int(re.findall(r'\d+', x.split('_')[-1].split('.')[0])[0]))  # Sort numerically
    return [(fits.open(file), file) for file in files]

def calibrate_and_save_images(science_images, flat_images, bias_images, output_folder):
    """Calibrate science images using provided flat and bias frames, perform cosmic ray removal, realign images, stack all images together, and save the calibrated images."""
    # Debug: Print the number of loaded bias and flat images
    print(f"Number of bias images: {len(bias_images)}")
    print(f"Number of flat images: {len(flat_images)}")
    
    bias_data = [CCDData(img[0].data, unit='adu') for img, _ in bias_images]
    combined_bias = ccdproc.combine(bias_data, method='median')

    flat_data = [CCDData(img[0].data, unit='adu') for img, _ in flat_images]
    corrected_flats = [ccdproc.subtract_bias(flat, combined_bias) for flat in flat_data]
    
    # Debug: Check if flat images are processed correctly
    if not corrected_flats:
        print("No corrected flat images found.")
    
    for flat in corrected_flats:
        cosmic_mask, clean_data = detect_cosmics(flat.data, gain=1.0, readnoise=10.0, sigclip=5.0)
        flat.data = clean_data

    normalized_flats = [flat.divide(np.median(flat.data[flat.data > 0])) for flat in corrected_flats]
    
    # Debug: Check if normalization was successful
    print(f"Number of normalized flats: {len(normalized_flats)}")
    if not normalized_flats:
        print("Normalization of flat images failed. Exiting.")
        return []
    
    combined_flat = ccdproc.combine(normalized_flats, method='median')

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    science_ccds = [CCDData(img[0].data, unit='adu', meta=img[0].header.copy()) for img, _ in science_images]

    corrected_science_ccds = []
    reference_image = None
    exposure_time = science_images[0][0][0].header['EXPTIME']
    print(f"Exposure time of each image: {exposure_time} seconds")

    stack_data = np.zeros_like(science_ccds[0].data, dtype='float32')
    stack_header = science_ccds[0].meta

    successful_stacks = 0
    failed_stacks = 0

    for (science_ccd_hdulist, filename) in science_images:
        try:
            # Ensure we are working with CCDData
            science_ccd = CCDData(science_ccd_hdulist[0].data, unit='adu', meta=science_ccd_hdulist[0].header.copy())

            corrected_ccd = ccdproc.subtract_bias(science_ccd, combined_bias)
            corrected_ccd = ccdproc.flat_correct(corrected_ccd, combined_flat)

            cosmic_mask, clean_data = detect_cosmics(corrected_ccd.data, gain=1.0, readnoise=10.0, sigclip=5.0, sigfrac=0.3)
            corrected_ccd.data = clean_data

            if reference_image is None:
                reference_image = corrected_ccd

            # Enhanced alignment process
            try:
                aligned_data, _ = aa.register(corrected_ccd.data, reference_image.data, max_control_points=150, detection_sigma=2.5, min_area=4)
            except Exception as alignment_error:
                print(f"Retrying alignment for image {filename} with adjusted parameters due to error: {alignment_error}")
                aligned_data, _ = aa.register(corrected_ccd.data, reference_image.data, max_control_points=100, detection_sigma=3.5, min_area=6)

            aligned_ccd = CCDData(aligned_data, unit='adu', meta=corrected_ccd.meta)
            stack_data += aligned_ccd.data
            successful_stacks += 1
        except Exception as e:
            print(f"Skipping image {filename} due to error: {e}")
            failed_stacks += 1

    if successful_stacks > 0:
        stack_data /= successful_stacks
        stacked_ccd = CCDData(stack_data.astype('float32'), unit='adu', meta=stack_header)
        corrected_science_ccds.append(stacked_ccd)

        output_file = os.path.join(output_folder, 'stacked_image.fits')
        fits.writeto(output_file, stacked_ccd.data, stacked_ccd.meta, overwrite=True)

    print(f"Number of successfully stacked images: {successful_stacks}")
    print(f"Number of images that failed to stack: {failed_stacks}")

    return corrected_science_ccds

# Example usage--------------------------------------------------------SET FOLDER HERE
science_folder = "D:\\Astronomy-Image-Processing\\m101\\m101_r"
flat_folder = "D:\\Astronomy-Image-Processing\\m101\\flat_r"
bias_folder = "D:\\Astronomy-Image-Processing\\m101\\bias"
output_folder = "D:\\Astronomy-Image-Processing\\m101\\calibrated_images_m101"

flat_images = load_all_fits_images(flat_folder)
bias_images = load_all_fits_images(bias_folder)
science_images = load_all_fits_images(science_folder)

corrected_science_ccds = calibrate_and_save_images(science_images, flat_images, bias_images, output_folder)
print("Calibration and stacking complete.")


In [9]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained ESRGAN model from TensorFlow Hub
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")

def enhance_image(image):
    return model(image)

# Load and preprocess your greyscale image
image = tf.image.decode_image(tf.io.read_file('path_to_your_greyscale_image.jpg'), channels=1)
image = tf.image.grayscale_to_rgb(image)  # Convert to 3 channels as required by the model
image = tf.expand_dims(image, axis=0)  # Add batch dimension

# Enhance the image
enhanced_image = enhance_image(image)

# Convert back to greyscale and save the enhanced image
enhanced_image = tf.image.rgb_to_grayscale(tf.squeeze(enhanced_image))
tf.io.write_file('enhanced_image.jpg', tf.image.encode_jpeg(enhanced_image))


NotFoundError: NewRandomAccessFile failed to Create/Open: path_to_your_greyscale_image.jpg : The system cannot find the file specified.
; No such file or directory [Op:ReadFile]